In [1]:
import pandas as pd
train=pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/train.csv')
train.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0


In [2]:
train['check']=train['healthy']+train['multiple_diseases']+ train['rust']+ train['scab']
train['check'].value_counts()                                                                

1    1821
Name: check, dtype: int64

In [3]:
train.drop(['check'],axis=1,inplace=True)
train.columns=['image_id', 'Target__healthy', 'Target__multiple_diseases',
               'Target__rust', 'Target__scab']
train.head()

,image_id,Target__healthy,Target__multiple_diseases,Target__rust,Target__scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0


In [4]:
def undummify(df, prefix_sep="__"):
    cols2collapse = {
        item.split(prefix_sep)[0]: (prefix_sep in item) for item in df.columns
    }
    series_list = []
    for col, needs_to_collapse in cols2collapse.items():
        if needs_to_collapse:
            undummified = (
                df.filter(like=col)
                .idxmax(axis=1)
                .apply(lambda x: x.split(prefix_sep, maxsplit=1)[1])
                .rename(col)
            )
            series_list.append(undummified)
        else:
            series_list.append(df[col])
    undummified_df = pd.concat(series_list, axis=1)
    return undummified_df

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1821 entries, 0 to 1820
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   image_id                   1821 non-null   object
 1   Target__healthy            1821 non-null   int64 
 2   Target__multiple_diseases  1821 non-null   int64 
 3   Target__rust               1821 non-null   int64 
 4   Target__scab               1821 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 71.3+ KB


In [6]:
train.nunique()

image_id                     1821
Target__healthy                 2
Target__multiple_diseases       2
Target__rust                    2
Target__scab                    2
dtype: int64

In [7]:
train=undummify(train)
train.head()

,image_id,Target
0,Train_0,scab
1,Train_1,multiple_diseases
2,Train_2,healthy
3,Train_3,rust
4,Train_4,healthy


In [8]:
train['filename']='/kaggle/input/plant-pathology-2020-fgvc7/images/'+train['image_id']+'.jpg'
train=train.drop(['image_id'],axis=1)
train.columns=['label','filename']
train.head()

,label,filename
0,scab,/kaggle/input/plant-pathology-2020-fgvc7/image...
1,multiple_diseases,/kaggle/input/plant-pathology-2020-fgvc7/image...
2,healthy,/kaggle/input/plant-pathology-2020-fgvc7/image...
3,rust,/kaggle/input/plant-pathology-2020-fgvc7/image...
4,healthy,/kaggle/input/plant-pathology-2020-fgvc7/image...


In [9]:
import os
import shutil
import pandas as pd

def copy_files(df, src_col_name, dest_col_name):
    """
    copys files from the source folder specified in the src_col_name column
    to the destination folder specified in the dest_col_name column of a pandas
    DataFrame.

    Args:
        df (pandas.DataFrame): The input DataFrame containing file locations.
        src_col_name (str): The name of the column containing the source file locations.
        dest_col_name (str): The name of the column containing the destination file locations.
    """
    for index, row in df.iterrows():
        src_path = row[src_col_name]
        dest_path = row[dest_col_name]
        if os.path.exists(src_path):
            if os.path.isfile(src_path):  # check if src_path is a file
                shutil.copy(src_path, dest_path)
            else:
                print(f"Cannot copy '{src_path}'. It is a directory.")
        else:
            print(f"Source path '{src_path}' does not exist.")


In [10]:
sol=pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/test.csv')
sol.head()

,image_id
0,Test_0
1,Test_1
2,Test_2
3,Test_3
4,Test_4


In [11]:
import shutil, os
newpath = r'./sol' 
if not os.path.exists(newpath):
    os.makedirs(newpath)

In [12]:
sol['filename']='/kaggle/input/plant-pathology-2020-fgvc7/images/'+sol['image_id']+'.jpg'
sol['destination']='./sol/'
sol.head()

,image_id,filename,destination
0,Test_0,/kaggle/input/plant-pathology-2020-fgvc7/image...,./sol/
1,Test_1,/kaggle/input/plant-pathology-2020-fgvc7/image...,./sol/
2,Test_2,/kaggle/input/plant-pathology-2020-fgvc7/image...,./sol/
3,Test_3,/kaggle/input/plant-pathology-2020-fgvc7/image...,./sol/
4,Test_4,/kaggle/input/plant-pathology-2020-fgvc7/image...,./sol/


In [13]:
copy_files(sol, 'filename', 'destination')

In [14]:
sol=sol[['image_id']]
sol.head()

,image_id
0,Test_0
1,Test_1
2,Test_2
3,Test_3
4,Test_4


In [15]:
sol.to_csv('./submission.csv',index=False)

In [16]:
from sklearn.model_selection import train_test_split
train,test= train_test_split( train, test_size=0.2, random_state=42,shuffle=True, stratify=train['label'])

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [17]:
count=(train.label.value_counts()).to_frame()
count.reset_index(inplace=True)
count

,index,label
0,rust,497
1,scab,473
2,healthy,413
3,multiple_diseases,73


In [18]:
train=train.sample(n=len(train))
train.reset_index(inplace=True,drop=True)
train.head()

,label,filename
0,healthy,/kaggle/input/plant-pathology-2020-fgvc7/image...
1,rust,/kaggle/input/plant-pathology-2020-fgvc7/image...
2,rust,/kaggle/input/plant-pathology-2020-fgvc7/image...
3,healthy,/kaggle/input/plant-pathology-2020-fgvc7/image...
4,rust,/kaggle/input/plant-pathology-2020-fgvc7/image...


In [19]:
test=test.sample(n=len(test))
test.reset_index(inplace=True,drop=True)
test.head()

,label,filename
0,healthy,/kaggle/input/plant-pathology-2020-fgvc7/image...
1,scab,/kaggle/input/plant-pathology-2020-fgvc7/image...
2,healthy,/kaggle/input/plant-pathology-2020-fgvc7/image...
3,scab,/kaggle/input/plant-pathology-2020-fgvc7/image...
4,healthy,/kaggle/input/plant-pathology-2020-fgvc7/image...


In [20]:
train.shape,test.shape

((1456, 2), (365, 2))

In [21]:
labels = train.sort_values('label')
labels.head()

,label,filename
0,healthy,/kaggle/input/plant-pathology-2020-fgvc7/image...
445,healthy,/kaggle/input/plant-pathology-2020-fgvc7/image...
444,healthy,/kaggle/input/plant-pathology-2020-fgvc7/image...
1005,healthy,/kaggle/input/plant-pathology-2020-fgvc7/image...
1006,healthy,/kaggle/input/plant-pathology-2020-fgvc7/image...


In [22]:
# take unique labels
class_names = list(labels.label.unique())
class_names[:5]

['healthy', 'multiple_diseases', 'rust', 'scab']

In [23]:
import os
newpath = r'./train' 
if not os.path.exists(newpath):
    os.makedirs(newpath)

In [24]:
import shutil, os
#creating subfolders
for c in class_names:
    dest =  r'./train/'+str(c)
    os.makedirs(dest)
    for i in list(labels[labels['label']==c]['filename']): # Image Id
        get_image = os.path.join('/kaggle/input/plant-pathology-2020-fgvc7/images', i) # Path to Images 
        try:
            x =move_image_to_cat = shutil.copy(get_image, dest)
        except FileNotFoundError as e:
            print(f"FileNotFoundError successfully handled\n"
                    f"{e}")

In [25]:
newpath = r'./test' 
if not os.path.exists(newpath):
    os.makedirs(newpath)

In [26]:
for c in class_names:
    dest =  r'./test/'+str(c)
    os.makedirs(dest)
    for i in list(test[test['label']==c]['filename']): # Image Id
        get_image = os.path.join('/kaggle/input/plant-pathology-2020-fgvc7/images', i) # Path to Images 
        try:
            x =move_image_to_cat = shutil.copy(get_image, dest)
        except FileNotFoundError as e:
            print(f"FileNotFoundError successfully handled\n"
                    f"{e}")